In [95]:
include("prac1.jl")

valdiff (generic function with 3 methods)

In [96]:
Base. mod(a::Tuple{Vararg{T}}, b::Tuple{Vararg{T}}) where T = begin
    p1 = Polynomial{T}([a[i] for i in range(1, length(a))])
    p2 = Polynomial{T}([b[i] for i in range(1, length(b))])
    return p1 % p2
end 

In [97]:
Base. mod(a::Polynomial{T}, b::T) where T = begin
    p = []
    for i in a.coeff
        append!(p, mod(i, b))
    end
    return Polynomial{T}(p)
end

In [98]:
struct Residue{T,M}
    value::T
    Residue{T,M}(val) where {T,M} = new(mod(val, M))
end

In [99]:
Base. +(a::Residue{T, M}, b::Residue{T, M}) where {T, M} = Residue{T, M}(a.value + b.value)
Base. -(a::Residue{T, M}, b::Residue{T, M}) where {T, M} = Residue{T, M}(a.value - b.value)
Base. -(a::Residue{T, M}) where {T, M} = Residue{T, M}(-a.value)
Base. *(a::Residue{T, M}, b::Residue{T, M}) where {T, M} = Residue{T, M}(a.value * b.value)
Base. ^(a::Residue{T, M}, b::T) where {T, M} = Residue{T, M}(a.value^b)
Base. one(::Type{Residue{T, M}}) where {T, M} = Residue{T, M}(Base.one(T))
Base. zero(::Type{Residue{T, M}}) where {T, M} = Residue{T, M}(Base.zero(T))

In [100]:
a = Residue{Int, 5}(12)

Residue{Int64, 5}(2)

In [101]:
b = Residue{Int, 5}(10)

Residue{Int64, 5}(0)

In [102]:
Residue{Polynomial{Int}, (1, 0, 0, 0)}((1, 0, 1, 1))

Residue{Polynomial{Int64}, (1, 0, 0, 0)}(Polynomial{Int64}([1, 1]))

In [103]:
Base. convert(::Type{Residue{T, M}}, a::T) where {T, M} = Residue{T, M}(a)

In [104]:
Polynomial{Residue{Int, 5}}([5, 6, 7])

Residue{Int64, 5}(0)x^2 + Residue{Int64, 5}(1)x^1 + Residue{Int64, 5}(2)

In [105]:
function gcdx_(a::T, b::T) where T
    u, v = Base. one(T), Base. zero(T)
    u_, v_ = v, u
    a0, b0 = a, b
    #Инвариант: НОД(a, b) = НОД(a0, b0) && a = u*a0 + v*b0 && b = u_*a0 + v_*b0
    while !iszero(b)
        k, r = divrem(a, b)
        a, b = b, r
        u, u_ = u_, u-k*u_
        v, v_ = v_, v-k*v_
    end
    if a < 0
        a, u, v = -a, -u, -v
    end
    return a, u, v
end

gcdx_ (generic function with 1 method)

In [106]:
function invmod(a::Residue{T, M}) where {T, M}
    a, u, v = gcdx_(a.value, M)
    if a != 1 return nothing end
    return Residue{T, M}(u)
end

invmod (generic function with 1 method)

In [107]:
invmod(Residue{Int, 10}(3))

Residue{Int64, 10}(7)

In [108]:
(7*3)%10

1

In [109]:
function diophant_solve(a::T, b::T, c::T) where T
    d, e, f = gcdx_(a, b)
    if d != c return nothing end
    return e, f
end

diophant_solve (generic function with 1 method)

In [110]:
diophant_solve(3, 10, 1)

(-3, 1)

In [111]:
function isprime(n::IntType) where IntType <: Integer
    for d in 2:IntType(ceil(sqrt(abs(n))))
        if n%d == 0
            return false
        end
    end
    return true
end

isprime (generic function with 1 method)

In [112]:
isprime(3)

true

In [113]:
isprime(4)

false

In [114]:
function eratosphenes_sieve(n::Integer)
    prime_indexes::Vector{Bool} = ones(Bool, n)
    prime_indexes[begin] = false
    i = 2
    prime_indexes[i^2:i:n] .= false
    i = 3
    #ИНВАРИАНТ: i - простое нечетное
    while i <= n
        prime_indexes[i^2:2i:n] .= false
        i += 1
        while i <= n && prime_indexes[i] == false
            i += 1
        end
    end
    return findall(prime_indexes)
end

eratosphenes_sieve (generic function with 1 method)

In [115]:
eratosphenes_sieve(10)

4-element Vector{Int64}:
 2
 3
 5
 7

In [126]:
function degree(n, p)
    k = 0
    n, r = divrem(n, p)
    while n > 0 && r == 0
        k += 1
        n, r = divrem(n, p)
    end
    return k
end

degree (generic function with 2 methods)

In [127]:
function factorize(n::T) where T <: Integer
    list = NamedTuple{(:div, :deg), Tuple{T, T}}[]
    for p in eratosphenes_sieve(Int(ceil(n/2)))
        k = degree(n, p)
        if k > 0
            push!(list, (div=p, deg=k))
        end
    end
    return list
end

factorize (generic function with 1 method)

In [128]:
factorize(10)

3-element Vector{@NamedTuple{div::Int64, deg::Int64}}:
 (div = 2, deg = 3)
 (div = 3, deg = 2)
 (div = 5, deg = 1)